<h1> TRABALHO PRATICO INTEGRADOR </h1>

<h2> Taxa de rotatividade em telecomunicação </h2>

<h4> 1. Limpesa dos dados</h4>

<h5> 1.1 Importação</h5>

In [185]:
import pandas as pd
import numpy as np

# Importando os dados

df = pd.read_csv("train.csv")

# Removendo coluna ID para evitar redundancia 

df = df.drop(['id'], axis = 1)
df

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,...,0,0,NaN,NaN,NaN,1958,0.00,0.00,0.00,0
1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,...,0,0,NaN,1.0,NaN,710,0.00,0.00,0.00,0
2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,0.000,...,0,0,NaN,NaN,NaN,882,0.00,0.00,0.00,0
3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,111.095,...,0,0,NaN,NaN,NaN,982,0.00,0.00,0.00,0
4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,101.565,...,1,0,1.0,1.0,1.0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,15.760,410.924,329.136,...,1,0,NaN,1.0,1.0,221,0.00,0.00,0.00,0
69995,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,160.083,289.129,265.772,...,0,0,NaN,NaN,NaN,712,0.00,0.00,0.00,0
69996,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,372.088,258.374,279.782,...,0,0,NaN,NaN,NaN,879,0.00,0.00,0.00,0
69997,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,238.575,245.414,145.062,...,0,0,1.0,1.0,1.0,277,664.25,1402.96,990.97,0


<h5> 1.2 Exploração</h5>

Dicionario do dataset:
<ul>
<li>CIRCLE_ID: Área a qual o cliente pertence</li>
<li>LOC: Ligações locais - dentro da mesma área</li>
<li>STD: chamadas padrão - fora da área</li>
<li>IC: Chamadas recebidas</li>
<li>OG: Chamadas realizadas</li>
<li>T2T: Ligações entre mesma operadora (entre celulares)</li>
<li>T2M: Ligações entre outra operadora de celular</li>
<li>T2O: Ligações para outra operadora de linha fixa</li>
<li>T2F: Ligações para linha fixa da mesma operadora</li>
<li>T2C: Ligações para o próprio call center</li>
<li>ARPU: Receita média por cliente</li>
<li>MOU: Minutos de uso - chamadas de voz</li>
<li>AON: Tempo na operadora - número de dias que o cliente está usando a operadora</li>
<li>ONNET: Todos os tipos de chamadas dentro da mesma operadora</li>
<li>OFFNET: Todos os tipos de chamadas de fora da operadora</li>
<li>ROAM: Indica que o cliente está na zona de roaming (itinerância) durante a chamada</li>
<li>SPL: Chamadas especiais</li>
<li>ISD: Chamadas internacionais</li>
<li>RECH: Recarga</li>
<li>NUM: Número</li>
<li>AMT: Valor em moeda local</li>
<li>MAX: Máximo</li>
<li>DATA: Internet móvel</li>
<li>3G: Network 3G</li>
<li>AV: Média</li>
<li>VOL: Volume de uso da internet móvel (em MB)</li>
<li>2G: Network 2G</li>
<li>PCK: Serviços pré-pagos</li>
<li>NIGHT: Serviços somente noturnos</li>
<li>MONTHLY: Serviços com validade equivalente a um mês</li>
<li>SACHET: Serviços com validade menor que um mês</li>
<li>*.6: KPI relativo ao mês de junho</li>
<li>*.7: KPI relativo ao mês de julho</li>
<li>*.8: KPI relativo ao mês de agosto</li>
<li>FB_USER: Serviço de benefício ao uso do Facebook e outras redes sociais similares</li>
<li>VBC: Custo baseado em volume - quando nenhum serviço específico é adquirido e</li>
pago por uso 

    
</ul>

In [186]:
#Verificando a quantidade de linhas e colunas
df.shape

(69999, 171)

In [187]:
df.columns

Index(['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou',
       'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8',
       'arpu_6', 'arpu_7', 'arpu_8',
       ...
       'sachet_3g_7', 'sachet_3g_8', 'fb_user_6', 'fb_user_7', 'fb_user_8',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'churn_probability'],
      dtype='object', length=171)

In [188]:
#Descrição estatistica dos daods
df.describe()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,66296.000000,...,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.153275,133.894438,132.978257,...,0.081444,0.085487,0.916325,0.909544,0.890319,1220.639709,68.108597,65.935830,60.07674,0.101887
std,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,299.963093,311.277193,311.896596,...,0.634547,0.680035,0.276907,0.286842,0.312501,952.426321,269.328659,267.899034,257.22681,0.302502
min,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,7.410000,6.675000,6.410000,...,0.000000,0.000000,1.000000,1.000000,1.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,34.110000,32.280000,32.100000,...,0.000000,0.000000,1.000000,1.000000,1.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,119.390000,115.837500,115.060000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,10752.560000,...,33.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


In [189]:
# Informações Gerais do data set
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 171 entries, circle_id to churn_probability
dtypes: float64(135), int64(27), object(9)
memory usage: 91.3+ MB


In [190]:
#Verificando os valores nulos do data set
df.isnull().sum()

circle_id                 0
loc_og_t2o_mou          702
std_og_t2o_mou          702
loc_ic_t2o_mou          702
last_date_of_month_6      0
                       ... 
aon                       0
aug_vbc_3g                0
jul_vbc_3g                0
jun_vbc_3g                0
churn_probability         0
Length: 171, dtype: int64

In [191]:
#Checando a porcentagem de valores nulos ou ausentes de cada coluna
df_percent = df.isna().sum()/df.isna().count()*100
print(df_percent.to_string())

circle_id                    0.000000
loc_og_t2o_mou               1.002871
std_og_t2o_mou               1.002871
loc_ic_t2o_mou               1.002871
last_date_of_month_6         0.000000
last_date_of_month_7         0.570008
last_date_of_month_8         1.047158
arpu_6                       0.000000
arpu_7                       0.000000
arpu_8                       0.000000
onnet_mou_6                  3.954342
onnet_mou_7                  3.838626
onnet_mou_8                  5.290076
offnet_mou_6                 3.954342
offnet_mou_7                 3.838626
offnet_mou_8                 5.290076
roam_ic_mou_6                3.954342
roam_ic_mou_7                3.838626
roam_ic_mou_8                5.290076
roam_og_mou_6                3.954342
roam_og_mou_7                3.838626
roam_og_mou_8                5.290076
loc_og_t2t_mou_6             3.954342
loc_og_t2t_mou_7             3.838626
loc_og_t2t_mou_8             5.290076
loc_og_t2m_mou_6             3.954342
loc_og_t2m_m

<h5> 1.3 Limpeza</h5>

Para valores de recarga total, que são as variaveis com maior porcentagem de valeros nulos, podemos considerarar que esses valores são igaus a zero, pois representa que não foi feita nenhuma recarga.


In [192]:
imput_nulos = ['total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 
'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8']

In [193]:
df[imput_nulos] = df[imput_nulos].apply(lambda x: x.fillna(0))

In [194]:
#Verificando que os valores foram realmente substituidos
df[imput_nulos].isnull().sum()

total_rech_data_6     0
total_rech_data_7     0
total_rech_data_8     0
av_rech_amt_data_6    0
av_rech_amt_data_7    0
av_rech_amt_data_8    0
max_rech_data_6       0
max_rech_data_7       0
max_rech_data_8       0
dtype: int64

In [195]:
# Categorizando cada coluna por coluna de ID, coluna de data, coluna de categoria e coluna numérica

column_id = ['circle_id']

columns_date = ['last_date_of_month_6',
             'last_date_of_month_7',
             'last_date_of_month_8',             
             'date_of_last_rech_6',
             'date_of_last_rech_7',
             'date_of_last_rech_8',             
             'date_of_last_rech_data_6',
             'date_of_last_rech_data_7',
             'date_of_last_rech_data_8']

columns_category = ['night_pck_user_6',
             'night_pck_user_7',
             'night_pck_user_8',             
             'fb_user_6',
             'fb_user_7',
             'fb_user_8']

columns_num = [column for column in df.columns if column not in column_id + columns_date + columns_category]

print("Colunas de Datas:%d\nColunas Numericas:%d\nColunas de categorias :%d" % (len(columns_date), len(columns_num), len(columns_category)))

# Verificando se classificamos todas as calunas

print(len(columns_date) + len(columns_num) + len(columns_category) + len(column_id) == df.shape[1])


Colunas de Datas:9
Colunas Numericas:155
Colunas de categorias :6
True


In [196]:
df[columns_date]

,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8
0,6/30/2014,7/31/2014,8/31/2014,6/22/2014,7/10/2014,8/24/2014,NaN,NaN,NaN
1,6/30/2014,7/31/2014,8/31/2014,6/12/2014,7/10/2014,8/26/2014,NaN,7/8/2014,NaN
2,6/30/2014,7/31/2014,8/31/2014,6/11/2014,7/22/2014,8/24/2014,NaN,NaN,NaN
3,6/30/2014,7/31/2014,8/31/2014,6/15/2014,7/21/2014,8/25/2014,NaN,NaN,NaN
4,6/30/2014,7/31/2014,8/31/2014,6/25/2014,7/26/2014,8/30/2014,6/25/2014,7/23/2014,8/20/2014
...,...,...,...,...,...,...,...,...,...
69994,6/30/2014,7/31/2014,8/31/2014,6/18/2014,7/31/2014,8/31/2014,NaN,7/31/2014,8/21/2014
69995,6/30/2014,7/31/2014,8/31/2014,6/28/2014,7/31/2014,8/27/2014,NaN,NaN,NaN
69996,6/30/2014,7/31/2014,8/31/2014,6/25/2014,7/30/2014,8/29/2014,NaN,NaN,NaN
69997,6/30/2014,7/31/2014,8/31/2014,6/29/2014,7/19/2014,8/26/2014,6/17/2014,7/13/2014,8/14/2014


Como datas específicas e o id da area não são dados relevantes vamos removelos do data set

In [197]:
df = df.drop(column_id + columns_date, axis=1)
df.shape

# Talves a área possa sim ser relevante, pois alguns planos podem funcionar melhor em uma região e em outra não. Segue o codigo
# pra remover só a data. basta remover o "#"

#df = df.drop(columns_date, axis=1)
#df.shape

(69999, 161)

In [198]:
df[columns_category]

,night_pck_user_6,night_pck_user_7,night_pck_user_8,fb_user_6,fb_user_7,fb_user_8
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...
69994,NaN,0.0,0.0,NaN,1.0,1.0
69995,NaN,NaN,NaN,NaN,NaN,NaN
69996,NaN,NaN,NaN,NaN,NaN,NaN
69997,0.0,0.0,0.0,1.0,1.0,1.0


In [199]:
#Substituindo os valores nulos na coluna de categoria por "-1"

df[columns_category] = df[columns_category].apply(lambda x: x.fillna(-1))

#Podemos também substituir por "0" no lugar de "-1", o que implicaria que o usuario definitivamente não utilizava determinados serviços.
#df[columns_category] = df[columns_category].apply(lambda x: x.fillna(0))
# basta remover o "#"

In [200]:
#Removendo variáveis com mais de 70% do valores nulos

initial_columns = df.shape[1]

#Para mudar a porcentagem limite basta mudar o "threshold", porém exite um padrão em que as featurea ou tem menos de 6% ou mais que
# 70% de seus valores nulos ou ausentes.

threshold = 0.7

include_columns = list(df.apply(lambda column: True if column.isnull().sum()/df.shape[0] < threshold else False))

drop_missing = pd.DataFrame({'features':df.columns , 'include': include_columns})
drop_missing.loc[drop_missing.include == True,:]


,features,include
0,loc_og_t2o_mou,True
1,std_og_t2o_mou,True
2,loc_ic_t2o_mou,True
3,arpu_6,True
4,arpu_7,True
5,arpu_8,True
6,onnet_mou_6,True
7,onnet_mou_7,True
8,onnet_mou_8,True
9,offnet_mou_6,True


In [201]:
df = df.loc[:, include_columns]
df.shape

(69999, 149)

In [202]:
# Substituindo o restante dos valores nulos pela mediana

columns_num = [column for column in df.columns if column not in column_id + columns_date + columns_category]

df[columns_num] = df[columns_num].apply(lambda x: x.fillna(x.median()))

#Substituindo pela média
#df[columns_num] = df[columns_num].apply(lambda x: x.fillna(x.mean()))

df[columns_num].isnull().sum()*100/df[columns_num].shape[0]

loc_og_t2o_mou        0.0
std_og_t2o_mou        0.0
loc_ic_t2o_mou        0.0
arpu_6                0.0
arpu_7                0.0
arpu_8                0.0
onnet_mou_6           0.0
onnet_mou_7           0.0
onnet_mou_8           0.0
offnet_mou_6          0.0
offnet_mou_7          0.0
offnet_mou_8          0.0
roam_ic_mou_6         0.0
roam_ic_mou_7         0.0
roam_ic_mou_8         0.0
roam_og_mou_6         0.0
roam_og_mou_7         0.0
roam_og_mou_8         0.0
loc_og_t2t_mou_6      0.0
loc_og_t2t_mou_7      0.0
loc_og_t2t_mou_8      0.0
loc_og_t2m_mou_6      0.0
loc_og_t2m_mou_7      0.0
loc_og_t2m_mou_8      0.0
loc_og_t2f_mou_6      0.0
loc_og_t2f_mou_7      0.0
loc_og_t2f_mou_8      0.0
loc_og_t2c_mou_6      0.0
loc_og_t2c_mou_7      0.0
loc_og_t2c_mou_8      0.0
loc_og_mou_6          0.0
loc_og_mou_7          0.0
loc_og_mou_8          0.0
std_og_t2t_mou_6      0.0
std_og_t2t_mou_7      0.0
std_og_t2t_mou_8      0.0
std_og_t2m_mou_6      0.0
std_og_t2m_mou_7      0.0
std_og_t2m_m

In [203]:
#Verificando se ainda há valores nulos no dataset

features_missing = df.isnull().sum()*100/df.shape[0]
print (features_missing.to_string())

loc_og_t2o_mou        0.0
std_og_t2o_mou        0.0
loc_ic_t2o_mou        0.0
arpu_6                0.0
arpu_7                0.0
arpu_8                0.0
onnet_mou_6           0.0
onnet_mou_7           0.0
onnet_mou_8           0.0
offnet_mou_6          0.0
offnet_mou_7          0.0
offnet_mou_8          0.0
roam_ic_mou_6         0.0
roam_ic_mou_7         0.0
roam_ic_mou_8         0.0
roam_og_mou_6         0.0
roam_og_mou_7         0.0
roam_og_mou_8         0.0
loc_og_t2t_mou_6      0.0
loc_og_t2t_mou_7      0.0
loc_og_t2t_mou_8      0.0
loc_og_t2m_mou_6      0.0
loc_og_t2m_mou_7      0.0
loc_og_t2m_mou_8      0.0
loc_og_t2f_mou_6      0.0
loc_og_t2f_mou_7      0.0
loc_og_t2f_mou_8      0.0
loc_og_t2c_mou_6      0.0
loc_og_t2c_mou_7      0.0
loc_og_t2c_mou_8      0.0
loc_og_mou_6          0.0
loc_og_mou_7          0.0
loc_og_mou_8          0.0
std_og_t2t_mou_6      0.0
std_og_t2t_mou_7      0.0
std_og_t2t_mou_8      0.0
std_og_t2m_mou_6      0.0
std_og_t2m_mou_7      0.0
std_og_t2m_m

In [204]:
df

,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0.0,0.0,0.0,31.277,87.009,7.527,48.58,124.38,1.29,32.24,...,0,0,-1.0,-1.0,-1.0,1958,0.00,0.00,0.00,0
1,0.0,0.0,0.0,0.000,122.787,42.953,0.00,0.00,0.00,0.00,...,0,0,-1.0,1.0,-1.0,710,0.00,0.00,0.00,0
2,0.0,0.0,0.0,60.806,103.176,0.000,0.53,15.93,0.00,53.99,...,0,0,-1.0,-1.0,-1.0,882,0.00,0.00,0.00,0
3,0.0,0.0,0.0,156.362,205.260,111.095,7.26,16.01,0.00,68.76,...,0,0,-1.0,-1.0,-1.0,982,0.00,0.00,0.00,0
4,0.0,0.0,0.0,240.708,128.191,101.565,21.28,4.83,6.13,56.99,...,1,0,1.0,1.0,1.0,647,0.00,0.00,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,0.0,0.0,0.0,15.760,410.924,329.136,0.00,7.36,10.93,0.00,...,1,0,-1.0,1.0,1.0,221,0.00,0.00,0.00,0
69995,0.0,0.0,0.0,160.083,289.129,265.772,116.54,196.46,232.63,49.53,...,0,0,-1.0,-1.0,-1.0,712,0.00,0.00,0.00,0
69996,0.0,0.0,0.0,372.088,258.374,279.782,77.13,68.44,78.44,335.54,...,0,0,-1.0,-1.0,-1.0,879,0.00,0.00,0.00,0
69997,0.0,0.0,0.0,238.575,245.414,145.062,14.01,7.64,6.71,30.34,...,0,0,1.0,1.0,1.0,277,664.25,1402.96,990.97,0


<h4> 2. </h4>